In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.io as pio
import math
from geojson_rewind import rewind
pio.renderers.default = 'iframe'

In [ ]:
def build_path(subfolder = 'figuras'):
    folderpath = os.path.join(os.getcwd(), subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath

def load_data():
    data = {}   
    folderpath = build_path()
    for filename in os.listdir(folderpath):
        if filename.endswith('.json'):
            filepath = os.path.join(build_path('figuras'), filename)
            data[filename.split('.')[0]] = pd.read_json(filepath)
    return data

def save_data(figure, type_of_map, intraregion, subfolder = 'app'):
    filepath = os.path.join(build_path(subfolder), f'{type_of_map}-{intraregion}.pkl')
    
    with open(filepath, 'wb') as pickle_file:
        pickle.dump(figure, pickle_file)      

In [ ]:

variaveis_uf_ano = pd.read_excel('./dados_para_dashboard/variaveis_uf_ano.xlsx', engine='openpyxl')



In [ ]:
# Criando UF e região
CondUF =  [(variaveis_uf_ano['uf'] == 'RO'),
         (variaveis_uf_ano["uf"] == 'AC'),
         (variaveis_uf_ano["uf"] == 'AM'),
         (variaveis_uf_ano["uf"] == 'RR'),
         (variaveis_uf_ano["uf"] == 'PA'),
         (variaveis_uf_ano["uf"] == 'AP'),
         (variaveis_uf_ano["uf"] == 'TO'),
         (variaveis_uf_ano["uf"] == 'MA'),
         (variaveis_uf_ano["uf"] == 'PI'),
         (variaveis_uf_ano["uf"] == 'CE'),
         (variaveis_uf_ano["uf"] == 'RN'),
         (variaveis_uf_ano["uf"] == 'PB'),
         (variaveis_uf_ano["uf"] == 'PE'),
         (variaveis_uf_ano["uf"] == 'AL'),
         (variaveis_uf_ano["uf"] == 'SE'),
         (variaveis_uf_ano["uf"] == 'BA'),
         (variaveis_uf_ano["uf"] == 'MG'),
         (variaveis_uf_ano["uf"] == 'ES'),
         (variaveis_uf_ano["uf"] == 'RJ'),
         (variaveis_uf_ano["uf"] == 'SP'),
         (variaveis_uf_ano["uf"] == 'PR'),
         (variaveis_uf_ano["uf"] == 'SC'),
         (variaveis_uf_ano["uf"] == 'RS'),
         (variaveis_uf_ano["uf"] == 'MS'),
         (variaveis_uf_ano["uf"] == 'MT'),
         (variaveis_uf_ano["uf"] == 'GO'),
         (variaveis_uf_ano["uf"] == 'DF')
         ]
          
UF_nomes = ['11','12','13','14','15','16','17','21','22','23','24','25','26','27','28','29','31','32','33','35','41','42','43','50','51','52','53']
variaveis_uf_ano["cod_UF"] = np.select(CondUF, UF_nomes, default="Error")

variaveis_uf_ano

In [ ]:
# gastospc uf

# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea']
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig = px.choropleth(variaveis_uf_ano,
                    geojson=geojson,
                    scope='south america',
                    color='gastos_percapita',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=5_00,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 25_000],  # Defina manualmente o intervalo de cores
                    locations='cod_UF',
                    featureidkey='properties.codarea',
                    hover_name='uf',
                    animation_frame='ano')

fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(marker_line_width=0)

fig.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 500, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])



In [3]:
%pip install streamlit_folium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
max=variaveis_uf_ano['gastos'].max()
print(max)

In [ ]:
min=variaveis_uf_ano['gastos'].min()
print(min)

In [ ]:
import pickle

def save_data(fig):
    arquivo = 'C:/Users/miche/Desktop/analise_gastos_saude/figuras/fig.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig, pickle_file)

In [ ]:
save_data(fig)

In [ ]:
# mapa gastos uf

# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea']
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig2 = px.choropleth(variaveis_uf_ano,
                    geojson=geojson,
                    scope='south america',
                    color='gastos',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=100_000,  # Defina um valor médio para o mapa de cores
                    range_color=[15_000_000, 6_000_000_000],  # Defina manualmente o intervalo de cores
                    locations='cod_UF',
                    featureidkey='properties.codarea',
                    hover_name='uf',
                    animation_frame='ano')

fig2.update_geos(fitbounds="locations", visible=False)
fig2.update_traces(marker_line_width=0)

fig2.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 500, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])



In [ ]:
def save_data(fig2):
    arquivo = 'C:/Users/miche/Desktop/analise_gastos_saude/figuras/fig2.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig2, pickle_file)

save_data(fig2)

In [ ]:
base_completa = pd.read_excel('./dados_tratados/base_completa.xlsx', engine='openpyxl')
base_completa


In [ ]:
# gastos municipio
# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig3 = px.choropleth(base_completa,
                    geojson=geojson,
                    scope='south america',
                    color='gastos',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=50_000,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 1_560_000_000],  # Defina manualmente o intervalo de cores
                    locations='cod_ibge',
                    featureidkey='properties.codarea',
                    hover_name='municipio',
                   animation_frame='ano')

fig3.update_geos(fitbounds="locations", visible=False)
fig3.update_traces(marker_line_width=0)

fig3.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 300, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig3.show()


In [ ]:
def save_data(fig3):
    arquivo = 'C:/Users/miche/Desktop/analise_gastos_saude/figuras/fig3.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig3, pickle_file)

save_data(fig3)

In [ ]:

# Puxar a malha geográfica do Brasil a nível de município
geojson = requests.get(f'http://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio').json()

# A malha geográfica do IBGE tem 7 dígitos (remover o dígito verificador e atualizar a malha)
for feature in geojson['features']:
    feature['properties']['codarea'] = feature['properties']['codarea'][:-1]
geojson = rewind(geojson, rfc7946=False)


# Construir o mapa choroplético com uma barra de cores fixa e sem variação
fig4 = px.choropleth(base_completa,
                    geojson=geojson,
                    scope='south america',
                    color='gastos_percapita',
                    color_continuous_scale="agsunset_r",  # Escolha um mapa de cores adequado aqui
                    color_continuous_midpoint=1_000,  # Defina um valor médio para o mapa de cores
                    range_color=[0, 3_700],  # Defina manualmente o intervalo de cores
                    locations='cod_ibge',
                    featureidkey='properties.codarea',
                    hover_name='municipio',
                   animation_frame='ano')

fig4.update_geos(fitbounds="locations", visible=False)
fig4.update_traces(marker_line_width=0)

fig4.update_layout(height=800, width=1000, autosize=False, 
                  updatemenus=[{"buttons": [{"args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "mode": "immediate", "transition": {"duration": 300, "easing": "linear"}}], "label": "Play", "method": "animate"}, 
                                          {"args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}], "label": "Stop", "method": "animate"}], 
                               "direction": "left", "pad": {"r": 10, "t": 87}, "showactive": False, "type": "buttons", "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"}])

fig4.show()


In [ ]:
def save_data(fig4):
    arquivo = 'C:/Users/miche/Desktop/analise_gastos_saude/figuras/fig4.pkl'
    with open(arquivo, 'wb') as pickle_file:
        pickle.dump(fig4, pickle_file)
save_data(fig4)

In [ ]:
import requests
url = 'https://github.com/heliomacedofilho/projetos-do-bootcamp-analise-de-dados-enap-2023/tree/main/automatizacao_de_relatorio_de_dados_de_documentos/analise_gastos_saude/figuras/'
response = requests.get(url)
response.content

In [ ]:
filename = 'gastos-uf.pkl'
import gzip

url = 'https://github.com/heliomacedofilho/projetos-do-bootcamp-analise-de-dados-enap-2023/tree/main/automatizacao_de_relatorio_de_dados_de_documentos/analise_gastos_saude/figuras/'
response = requests.get(f'{url}{filename}',stream='True')

with gzip.open(response.raw, 'rb') as ifp:
    print(pickle.load(ifp))
    
#pickle.load(response.raw, encoding='latin1')


In [ ]:
import streamlit as st
import pandas as pd
import requests
import pickle

# @st.cache(allow_output_mutation=True)
st.set_page_config(page_title="Municípios", page_icon="🏥", layout="wide")

st.title('_Municípios_')
st.markdown("---")

filename = 'fig.pkl'

url = 'https://github.com/heliomacedofilho/projetos-do-bootcamp-analise-de-dados-enap-2023/raw/main/automatizacao_de_relatorio_de_dados_de_documentos/analise_gastos_saude/figuras/'
response = requests.get(f'{url}{filename}', stream='True')
pickle.load(response.raw)